### Causal Interpretation for Ames Housing Price

This notebook uses the Ames Housing dataset to showcase how we can interpret a blackbox model from both the correlation and causation perspective by leveraging the power of model interpretation tools like SHAP and EconML. This housing dataset collects house prices and characteristics for homes sold in Ames, Iowa from the period of 2006 to 2010. We start with a linear regression to build intuition. We then train a fine-tuned predictive ML model and use SHAP to better understand the correlations between features and target and which features are the strongest predictors. Finally, we train a separate causal model using EconML, which identifies features that have a direct causal effect on housing price, instead of just predicting the housing price given a set of characteristics.

Note: A previous version of this notebook used the Boston Housing dataset. Due to ethical concerns with the Boston Housing dataset, we have decided to redo the analysis on the similar Ames Housing dataset. More details can be found on the sklearn page for the Boston Housing dataset.

In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import shap

### Linear Regression

In [2]:
from sklearn.datasets import fetch_openml

ames_housing = fetch_openml(name = 'house_prices', as_frame = True)

/opt/anaconda3/envs/FinancialMachineLearning/lib/python3.10/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
ames_df = ames_housing.data
print(ames_df.shape)

(1460, 80)


#### Data Preprocessing

Minor feature engineering via feature creation/removal, and outlier removal

In [4]:
Xy = (
    ames_df
    .assign(SalePrice = ames_housing.target) # add target feature
    .set_index('Id')
    .loc[
        lambda df: df['MasVnrType'].notna() # drop outliers with missing detail in this column
    ]
    .loc[
        lambda df: df['Electrical'].notna() # drop outlier with missing electrical row
    ]
    .assign(
        AgeAtSale = lambda df: df['YrSold'].sub(df['YearBuilt']), # add interpretable year columns
        YearsSinceRemodel = lambda df: df['YrSold'].sub(df['YearRemodAdd']).clip(lower = 0), # clip lower for outlier

        HasDeck = lambda df: df['WoodDeckSF'].gt(0).map(int),
        HasPorch = lambda df:
        df[['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']]
        .gt(0)
        .max(axis = 1)
        .map(int),

        HasFireplace = lambda df: df['Fireplaces'].clip(upper = 1).map(int),
        HasFence = lambda df: df['Fence'].notna().map(int)
    )

    # drop year columns
    .drop(
        columns = [
            'GarageYrBlt', 'YearBuilt', 'YrSold', 'YearRemodAdd',
            'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch',
            'FireplaceQu', 'Fireplaces',
            'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
            '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GarageArea', 'PoolArea'
        ]
    )
    .assign(LotFrontage = lambda df: df['LotFrontage'].fillna(0)) # fill missing with 0
    .fillna('NA') # rest of missing values are in categorical columns, so fill with NA category
    .assign(Intercept = 1) # add constant column for OLS
)

In [5]:
Xy

,MSSubClass,MSZoning,LotFrontage,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,SaleType,SaleCondition,SalePrice,AgeAtSale,YearsSinceRemodel,HasDeck,HasPorch,HasFireplace,HasFence,Intercept
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,...,WD,Normal,208500,5,5,0,1,0,0,1
2,20,RL,80.0,Pave,NA,Reg,Lvl,AllPub,FR2,Gtl,...,WD,Normal,181500,31,31,1,0,1,0,1
3,60,RL,68.0,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,...,WD,Normal,223500,7,6,0,1,1,0,1
4,70,RL,60.0,Pave,NA,IR1,Lvl,AllPub,Corner,Gtl,...,WD,Abnorml,140000,91,36,0,1,1,0,1
5,60,RL,84.0,Pave,NA,IR1,Lvl,AllPub,FR2,Gtl,...,WD,Normal,250000,8,8,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,62.0,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,...,WD,Normal,175000,8,7,0,1,1,0,1
1457,20,RL,85.0,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,...,WD,Normal,210000,32,22,1,0,1,1,1
1458,70,RL,66.0,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,...,WD,Normal,266500,69,4,0,1,1,1,1


Identify categorical columns for one hot encoding

In [7]:
categorial = list(
    Xy.apply(
        lambda series : series.dtype
    ).loc[
        lambda df : df.eq('object')
    ].index
) + ['MSSubClass']

In [10]:
X = Xy.drop(['SalePrice'], axis = 1)
X_ohe = X.pipe(
    pd.get_dummies, prefix_sep = '_OHE_', columns = categorial, dtype = 'uint8'
)
y = Xy['SalePrice'] # target

Train a linear regression using statsmodels

In [11]:
model = sm.OLS(y, X_ohe).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.923
Model:                            OLS   Adj. R-squared:                  0.908
Method:                 Least Squares   F-statistic:                     59.41
Date:                Tue, 22 Oct 2024   Prob (F-statistic):               0.00
Time:                        00:48:36   Log-Likelihood:                -16565.
No. Observations:                1451   AIC:                         3.362e+04
Df Residuals:                    1206   BIC:                         3.491e+04
Df Model:                         244                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
LotFrontage           